# CFEM - Itabira

## 1. Coleta de Dados

### 1.1 Conexão

In [223]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import json
from bs4 import BeautifulSoup
import os


driver = webdriver.Chrome()
url_paginaInicial = "https://servicositabira.govbr.cloud/pronimtb/"
url_despesas_fonteDeRecurso = "https://servicositabira.govbr.cloud/pronimtb/index.asp?acao=3&item=6"

driver.get(url_despesas_fonteDeRecurso) # Pagina Inicial > Despesas > Fonte de Recurso

# Esperar até que o DOM esteja carregado
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


<selenium.webdriver.remote.webelement.WebElement (session="4dfb2627613682e858b8a53eaeee132a", element="f.FA5081A97FF78ABEB3E09D6EF87E3928.d.C66F3BBA53E2D2F6616EEBA686979077.e.250")>

### 1.2 Filtros

In [224]:
# Data inicial
data_inicial = driver.find_element(By.ID, "txtDataInicial")
data_inicial.clear()
data_inicial.send_keys("01012024")

# Data final 
data_final = driver.find_element(By.ID, "txtDataFinal")
data_final.clear()
data_final.send_keys("31012024")


# Unidade Gestora
select_element = driver.find_element(By.ID, 'cmbUnidadeGestora')
select = Select(select_element)
select.select_by_value('0')  # Selecionar a opção com valor '0'

# Clicar no botão "Gerar"
element = driver.find_element(By.ID, "confirma")
element.click()

### 1.3 Função ler_tabela()

In [225]:
import json
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def ler_tabela(nome_dados):
    global dados_tabelas

    html_table = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
    
    # Passar o objeto html_table para BeautifulSoup
    soup = BeautifulSoup(html_table.get_attribute('outerHTML'), 'html.parser')
    
    # Find the table element
    table = soup.find('table')
    
    # Initialize an empty dictionary to store the table data
    table_data = {}
    
    # Extract the headers from the table
    header_row = table.find_all('tr')[1]  # Assuming the second row is the header
    headers = header_row.find_all('th')
    headers_text = [header.text.strip().replace('.', '') for header in headers]  # Remove dots from headers
    
    # Extract the data from the table
    for row in table.find_all('tr')[2:]:  # Starting from the third row to skip the header and total row
        cells = row.find_all('td')
        item = cells[0].get_text().strip().replace('.', '')  # Remove dots from item
        # Remove all dots from the text data
        cells_text = [cell.get_text().strip().replace('.', '') for cell in cells[1:]]
        data = {
            headers_text[i+1]: cells_text[i] for i in range(len(cells_text))
        }
        table_data[item] = data

    # Adiciona os dados ao dicionário global
    if nome_dados not in dados_tabelas:
        dados_tabelas[nome_dados] = table_data
    else:
        dados_tabelas[nome_dados].update(table_data)
    
    # Convert the data to a JSON string with indentation
    json_data = json.dumps(table_data, indent=4, ensure_ascii=False)
    
    # Define the directory and file name
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    # Save the JSON data to a file
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)
    print(f'Dados salvos em: {file_path}')


### 1.4 Seleção de Recursos

1.4.1 CFEM

In [226]:
# Selecionando CFEM na Tabela # Pagina Inicial > Despesas > Fonte de Recurso > CFEM
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
try:
    # Localize o link pelo texto
    link_cfem = driver.find_element(By.LINK_TEXT, "Transf União Referente à Compensação Financeira de Recursos Minerais")
    
    # Clique no link
    link_cfem.click()

except TimeoutException:
    print("Ocorreu um timeout ao esperar por um elemento.")
except NoSuchElementException:
    print("Não foi possível encontrar um elemento.")


1.4.2 Detalhamento da fonte de despesa (CFEM Ordinário)

In [227]:
#ler_tabela("desp_por_fonte")

In [228]:
# Selecionando Recurso Ordinário - CFEM na Tabela # Pagina Inicial > Despesas > Fonte de Recurso > CFEM > Recurso Ordinário
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
try:
    # Localize o link pelo texto
    link_recurso_ord_cfem = driver.find_element(By.LINK_TEXT, "000.0911 - Recurso Ordinário - CFEM")
    
    # Clique no link
    link_recurso_ord_cfem.click()

except TimeoutException:
    print("Ocorreu um timeout ao esperar por um elemento.")
except NoSuchElementException:
    print("Não foi possível encontrar um elemento.")

### 2.1 Ler tabela e salvar links numa lista

In [229]:
#ler_tabela("naturezas_despesas")

In [230]:
desc_natureza_despesa:str = '4.4.90.51 - Obras e Instalações'
def seleciona_natureza_despesa(id_tabela, descrição_natureza):
  
  wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


  table = driver.find_element(By.ID, id_tabela)

  # links = set()   ---> garante que os links sejam unicos
  links = []  # Lista garante a ordem de inserção dos links
  rows = table.find_elements(By.TAG_NAME, 'tr')
  for row in rows:
    try:
        link = row.find_element(By.XPATH, './td[1]//a') # encontrar um link dentro da primeira coluna
        href = link.get_attribute('href')
        links.append(href)  # Add para set / append para list
    except:
        continue
    #for link in links:
    #print(link)

  link_cfem_obras = driver.find_element(By.LINK_TEXT, descrição_natureza)
  link_cfem_obras.click()

    
seleciona_natureza_despesa('tbTabela',desc_natureza_despesa)

In [231]:
#ler_tabela("credores_obras_e_instalacoes")

In [232]:
def ler_tabela_credores(nome_dados):
    global dados_tabelas

    # Aguarda até que o elemento da tabela esteja presente na página
    html_table = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
    
    # Passa o objeto html_table para o BeautifulSoup
    soup = BeautifulSoup(html_table.get_attribute('outerHTML'), 'html.parser')
    
    # Encontra o elemento da tabela
    table = soup.find('table')

    table = soup.find('table', {'id': 'tbTabela'})

    # Extraí os cabeçalhos
    headers = [header.get_text().strip() for header in table.find_all('th')]
    #print(headers)
    # Remove possíveis pontos nos headers
    headers_text = [header.replace('.', '') for header in headers]
    #print(headers_text[1])
    # Dicionário para armazenar os dados da tabela
    table_data = {}

    # Extrai os dados da tabela, somente das linhas cujo ID contenha "parLinha"
    for row in table.find_all('tr')[1:]:  # Começa a partir da segunda linha para pular o cabeçalho
        if "parLinha" in row.get('id', ''):  # Verifica se "parLinha" está no ID da linha
            cells = row.find_all('td')
            
            if len(cells) > 0:
                item = cells[0].get_text().strip().replace('.', '')  # Remove pontos do item
                # Remove todos os pontos dos dados de texto
                cells_text = [cell.get_text().strip().replace('.', '') for cell in cells]
                data = {
                    headers_text[i+1]: cells_text[i] for i in range(len(cells_text))
                }
                table_data[item] = data

    # Adiciona os dados ao dicionário global
    if nome_dados not in dados_tabelas:
        dados_tabelas[nome_dados] = table_data
    else:
        dados_tabelas[nome_dados].update(table_data)

    # # Exibe os dados extraídos
    # for item, data in table_data.items():
    #     print(f"Item: {item}")
    #     for key, value in data.items():
    #         print(f"  {key}: {value}")


    # Convert the data to a JSON string with indentation
    json_data = json.dumps(table_data, indent=4, ensure_ascii=False)
    
    # Define the directory and file name
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    # Save the JSON data to a file
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)
    print(f'Dados salvos em: {file_path}')

    #driver.back()

ler_tabela_credores("credores")

Dados salvos em: json_output\credores.json


In [233]:
def ler_tabela_empenhos(nome_dados):
    global dados_tabelas

    # Aguarda até que o elemento da tabela esteja presente na página
    html_table = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
    
    # Passa o objeto html_table para o BeautifulSoup
    soup = BeautifulSoup(html_table.get_attribute('outerHTML'), 'html.parser')
    
    # Encontra o elemento da tabela
    table = soup.find('table')

    table = soup.find('table', {'id': 'tbTabela'})

    # Extraí os cabeçalhos
    headers = [header.get_text().strip() for header in table.find_all('th')]
    #print(headers)
    # Remove possíveis pontos nos headers
    headers_text = [header.replace('.', '') for header in headers]
    #print(headers_text[1])
    # Dicionário para armazenar os dados da tabela
    table_data = {}

    # Extrai os dados da tabela, somente das linhas cujo ID contenha "parLinha"
    for row in table.find_all('tr')[1:]:  # Começa a partir da segunda linha para pular o cabeçalho
        if "parLinha" in row.get('id', ''):  # Verifica se "parLinha" está no ID da linha
            cells = row.find_all('td')
            
            if len(cells) > 0:
                item = cells[1].get_text().strip().replace('.', '')  # Remove pontos do item
                # Remove todos os pontos dos dados de texto
                cells_text = [cell.get_text().strip().replace('.', '') for cell in cells]
                data = {
                    headers_text[i+1]: cells_text[i] for i in range(len(cells_text))
                }
                table_data[item] = data

    # Adiciona os dados ao dicionário global
    if nome_dados not in dados_tabelas:
        dados_tabelas[nome_dados] = table_data
    else:
        dados_tabelas[nome_dados].update(table_data)

    # # Exibe os dados extraídos
    # for item, data in table_data.items():
    #     print(f"Item: {item}")
    #     for key, value in data.items():
    #         print(f"  {key}: {value}")


    # Convert the data to a JSON string with indentation
    json_data = json.dumps(table_data, indent=4, ensure_ascii=False)
    
    # Define the directory and file name
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    # Save the JSON data to a file
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)
    print(f'Dados salvos em: {file_path}')

    #driver.back()

ler_tabela_empenhos("teste_empenhos")

Dados salvos em: json_output\teste_empenhos.json


In [234]:
# def clicar_credores_ler_datas(driver, tabela_id):

#     wait = WebDriverWait(driver, 10)
#     wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
#     table = driver.find_element(By.ID, tabela_id)
#     # Extrair os links da primeira coluna
#     rows = table.find_elements(By.TAG_NAME, 'tr')
    
#     for index, val in enumerate(rows):
#         if index == 0 or index == 1: continue
#         try:
#             table = driver.find_element(By.ID, tabela_id)
#             rows = table.find_elements(By.TAG_NAME, 'tr')
#             link = rows[index].find_element(By.XPATH, './td[1]/a')
#             link.click()


#             ler_tabela_datas(f"empenhos_credores_{index-1}")
#             driver.back()
#             WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, tabela_id)))
#             time.sleep(2)
#         except Exception as err:  
#             print(err)

# #clicar_credores_ler_datas(driver,'tbTabela')

In [235]:
# def clicar_linhas_primeira_coluna(driver, tabela_id, subindex=1):
#     wait = WebDriverWait(driver, 10)
#     wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
#     table = driver.find_element(By.ID, tabela_id)
#     # Extrair os links da primeira coluna
#     rows = table.find_elements(By.TAG_NAME, 'tr')
    
#     for index, val in enumerate(rows):
#         if index == 0 or index == 1: continue
#         try:
#             table = driver.find_element(By.ID, tabela_id)
#             rows = table.find_elements(By.TAG_NAME, 'tr')   
#             link = rows[index].find_element(By.XPATH, './td[1]/a') #/html/body/div/div[9]/table[4]/tbody/tr[3]/td[1]/a
#             link.click()
            
#             # Chama a função de forma recursiva com subindex incrementado
#             clicar_linhas_primeira_coluna(driver, tabela_id, subindex + 1)
#             ler_tabela_datas(f"empenhos_credores_{index-1}_{subindex}")


#             clicar_linhas_primeira_coluna(driver, tabela_id, subindex + 2)
#             ler_tabela(f"empenhos_credores_{index-1}_{subindex}")

#             driver.back()
#             WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, tabela_id)))
#             time.sleep(2)
            
            
#         except Exception as err:  
#             print(err)

# # Chamada inicial da função
# clicar_linhas_primeira_coluna(driver, 'tbTabela')


In [236]:
import json
import os
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Dicionário global para armazenar os dados
dados_tabelas = {}

def ler_descricao_empenho(nome_dados):
    global dados_tabelas

    # Coleta os dados da tabela tbHistorico
    try:
        html_historico = wait.until(EC.presence_of_element_located((By.ID, 'tbHistorico')))
        soup_historico = BeautifulSoup(html_historico.get_attribute('outerHTML'), 'html.parser')
        table_historico = soup_historico.find('table', {'id': 'tbHistorico'})

        headers_historico = [header.get_text().strip().replace('.', '') for header in table_historico.find_all('th')]

        historico_data = {}
        for row in table_historico.find_all('tr')[1:]:
            cells = row.find_all('td')
            if len(cells) > 0:
                item = cells[0].get_text().strip().replace('.', '')
                cells_text = [cell.get_text().strip().replace('.', '') for cell in cells]
                data = {
                    headers_historico[i]: cells_text[i] for i in range(len(cells_text))
                }
                historico_data[item] = data

        # Adiciona os dados ao dicionário global
        if nome_dados not in dados_tabelas:
            dados_tabelas[nome_dados] = {'historico': historico_data}
        else:
            dados_tabelas[nome_dados]['historico'] = historico_data

    except Exception as e:
        print(f"Erro ao coletar dados de tbHistorico: {e}")

    # Coleta os dados da tabela tbTabela
    try:
        html_tabela = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
        soup_tabela = BeautifulSoup(html_tabela.get_attribute('outerHTML'), 'html.parser')
        table_tabela = soup_tabela.find('table', {'id': 'tbTabela'})

        headers_tabela = [header.get_text().strip().replace('.', '') for header in table_tabela.find_all('th')]

        tabela_data = {}
        for row in table_tabela.find_all('tr')[1:]:
            cells = row.find_all('td')
            if len(cells) > 0:
                item = cells[0].get_text().strip().replace('.', '')
                cells_text = [cell.get_text().strip().replace('.', '') for cell in cells]
                data = {
                    headers_tabela[i]: cells_text[i] for i in range(len(cells_text))
                }
                tabela_data[item] = data


        print(tabela_data)

        # Adiciona os dados ao dicionário global
        if nome_dados not in dados_tabelas:
            dados_tabelas[nome_dados] = {'tabela': tabela_data}
        else:
            dados_tabelas[nome_dados]['tabela'] = tabela_data

    except Exception as e:
        print(f"Erro ao coletar dados de tbTabela: {e}")


    # Convert the data to a JSON string with indentation
    json_data = json.dumps(dados_tabelas, indent=4, ensure_ascii=False)
    
    # Define the directory and file name
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    # Save the JSON data to a file
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)
    print(f'Dados salvos em: {file_path}')

In [237]:
ler_descricao_empenho("teste_desc")

{'COMPLETA ENGENHARIA S/A': {'Credores': 'COMPLETA ENGENHARIA S/A', 'Nome': '**530446/0001-**', 'CNPJ/CPF': 'R$ 1407970,41', 'Valor Empenhado': 'R$ 0,00', 'Valor Em Liquidação': 'R$ 0,00', 'Valor Liquidado': 'R$ 0,00', 'Valor Pago': 'R$ 0,00'}, 'CONSTRUTORA APICE LTDA': {'Credores': 'CONSTRUTORA APICE LTDA', 'Nome': '**745428/0001-**', 'CNPJ/CPF': 'R$ 250236,59', 'Valor Empenhado': 'R$ 0,00', 'Valor Em Liquidação': 'R$ 0,00', 'Valor Liquidado': 'R$ 0,00', 'Valor Pago': 'R$ 0,00'}, 'CONSTRUTORA LINHARES LTDA': {'Credores': 'CONSTRUTORA LINHARES LTDA', 'Nome': '**272427/0001-**', 'CNPJ/CPF': 'R$ 757485,15', 'Valor Empenhado': 'R$ 0,00', 'Valor Em Liquidação': 'R$ 0,00', 'Valor Liquidado': 'R$ 0,00', 'Valor Pago': 'R$ 0,00'}, 'CONSTRUTORA NATIVA LTDA - EPP': {'Credores': 'CONSTRUTORA NATIVA LTDA - EPP', 'Nome': '**899663/0001-**', 'CNPJ/CPF': 'R$ 1,00', 'Valor Empenhado': 'R$ 0,00', 'Valor Em Liquidação': 'R$ 0,00', 'Valor Liquidado': 'R$ 0,00', 'Valor Pago': 'R$ 0,00'}, 'CONSTRUTORA UNIA

In [239]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def clicar_recursivo(driver, tabela_id):
    processed_rows = set()  # Set para rastrear linhas processadas
    wait = WebDriverWait(driver, 10)

    def encontrar_e_clicar_links(driver, tabela_id, coluna=1):
        while True:
            # Atualizar o conteúdo da tabela
            wait.until(EC.presence_of_element_located((By.ID, tabela_id)))
            table_html = driver.find_element(By.ID, tabela_id).get_attribute('outerHTML')
            soup = BeautifulSoup(table_html, 'html.parser')
            rows = soup.find_all('tr')

            new_links_found = False

            for index, row in enumerate(rows):
                if index < 2 or index in processed_rows:
                    continue  # Pular cabeçalho e linhas já processadas

                processed_rows.add(index)  # Marcar a linha como processada

                link = row.find_all('td')[coluna-1].find('a')
                if link is None:
                    continue
                link_href = link.get('href')  # Obter o atributo href do link

                try:
                    # Use JavaScript to navigate to avoid StaleElementReferenceException
                    driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, f'//a[@href="{link_href}"]'))
                    new_links_found = True

                    wait.until(EC.presence_of_element_located((By.ID, tabela_id)))

                    if coluna == 1:
                        nome_dados = f"empenhos_{index-1}"
                        # Lógica para processar a tabela de empenhos
                        # Exemplo: ler_tabela_empenhos(nome_dados)
                        encontrar_e_clicar_links(driver, tabela_id, 2)
                    else:
                        nome_dados = f"desc_{index-1}"
                        # Lógica para processar a descrição do empenho
                        # Exemplo: ler_descricao_empenho(nome_dados)

                    driver.back()
                    wait.until(EC.presence_of_element_located((By.ID, tabela_id)))
                except Exception as err:
                    print(err)
                    processed_rows.remove(index)  # Se falhar, remover a linha dos processados para tentar novamente

            if not new_links_found:
                break

    encontrar_e_clicar_links(driver, tabela_id)

# Exemplo de uso:
# Substitua 'driver' pelo seu WebDriver atual e certifique-se de que você já navegou até a página desejada
clicar_recursivo(driver, 'tbTabela')


{'Contratação de empresa para execução de serviços de obra de drenagem pluvial através de execução de mini túnel de concreto pelo método não destrutivo no bairro Fênix - Itabira/MG': {'Descrição do Empenho': 'Contratação de empresa para execução de serviços de obra de drenagem pluvial através de execução de mini túnel de concreto pelo método não destrutivo no bairro Fênix - Itabira/MG', 'Item': '-', 'Quantidade': '-', 'Unidade': '-', 'Valor Unitário': '-'}, 'CONTRATACAO DE OBRAS E SERVICOS': {'Descrição do Empenho': 'CONTRATACAO DE OBRAS E SERVICOS', 'Item': '1,0000', 'Quantidade': 'SV', 'Unidade': 'R$ 500000,00', 'Valor Unitário': 'R$ 500000,00'}}
Dados salvos em: json_output\desc_2.json
{'CONTRATACAO DE OBRAS E SERVICOS': {'Descrição do Empenho': 'CONTRATACAO DE OBRAS E SERVICOS', 'Item': '1,0000', 'Quantidade': 'SV', 'Unidade': 'R$ 307970,41', 'Valor Unitário': 'R$ 307970,41'}, 'Obra de drenagem pluvial na lateral da avenida France de Paula Andrade, entre ruas J e Totonho Ramos, bai

AttributeError: 'NoneType' object has no attribute 'get'

# Formatando Dados

In [133]:
import json
from bson.decimal128 import Decimal128

# Função para converter valores monetários
def converter_valores(json_obj, nome_arquivo):
    for key, value in json_obj.items():
        if isinstance(value, dict):
            converter_valores(value, nome_arquivo)  # Chamar recursivamente com o mesmo nome do arquivo
        else:
            if isinstance(value, str) and value.startswith("R$"):
                # Remover "R$ " e converter para float
                valor_convertido = value.replace("R$ ", "").replace(".", "").replace(",", ".")
                # Atualizar valor no JSON como Decimal128
                json_obj[key] = str(Decimal128(valor_convertido))

### Despesas por Fonte

In [134]:
# Nome do arquivo
nome_arquivo = "desp_por_fonte.json"

# Ler o arquivo JSON
with open(f"json_output/{nome_arquivo}", 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Converter os valores no JSON, passando o nome do arquivo como parâmetro
converter_valores(json_data, nome_arquivo)

# Salvar o JSON atualizado de volta no arquivo
with open(f"json_output/{nome_arquivo}", 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print("Arquivo atualizado com sucesso.")

Arquivo atualizado com sucesso.


### Natureza das Despesas

In [135]:
# Nome do arquivo
nome_arquivo = "naturezas_despesas.json"

# Ler o arquivo JSON
with open(f"json_output/{nome_arquivo}", 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Converter os valores no JSON, passando o nome do arquivo como parâmetro
converter_valores(json_data, nome_arquivo)

# Salvar o JSON atualizado de volta no arquivo
with open(f"json_output/{nome_arquivo}", 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print("Arquivo atualizado com sucesso.")

Arquivo atualizado com sucesso.


### Credores referentes à Obras e Instalações

In [136]:
# Nome do arquivo
nome_arquivo = "credores_obras_e_instalacoes.json"

# Ler o arquivo JSON
with open(f"json_output/{nome_arquivo}", 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Converter os valores no JSON, passando o nome do arquivo como parâmetro
converter_valores(json_data, nome_arquivo)

# Salvar o JSON atualizado de volta no arquivo
with open(f"json_output/{nome_arquivo}", 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print("Arquivo atualizado com sucesso.")

Arquivo atualizado com sucesso.
